In [4]:
import numpy as np
from flow_decomposer import FlowDecomposition

In [ ]:


#############################################
# New test example using mixed Lorenz systems #
#############################################

def simulate_lorenz(initial, sigma=10, rho=28, beta=8/3, dt=0.01, steps=1000):
    """
    Simulate a Lorenz attractor using Euler integration.
    Returns an array of shape (steps, 3).
    """
    trajectory = np.empty((steps, 3))
    trajectory[0] = initial
    for i in range(1, steps):
        x, y, z = trajectory[i-1]
        dx = sigma * (y - x)
        dy = x * (rho - z) - y
        dz = x * y - beta * z
        trajectory[i] = trajectory[i-1] + dt * np.array([dx, dy, dz])
    return trajectory

def test_flow_decomposition():
    steps = 10000

    # Simulate three independent Lorenz attractors.
    traj1 = simulate_lorenz(initial=[1.0, 1.0, 1.0], steps=steps)
    traj2 = simulate_lorenz(initial=[-1.0, -1.0, 1.0], steps=steps)
    traj3 = simulate_lorenz(initial=[0.5, 0.5, 0.0], steps=steps)
    
    # Stack the three trajectories horizontally.
    # This yields a latent data matrix of shape (steps, 9) [3 systems x 3 dimensions each].
    X_latent = np.hstack([traj1, traj2, traj3])
    
    # Create a mixing matrix with rank 2 so that, although we start with 3 Lorenz systems,
    # the effective number of independent dynamical systems in the observables is 2.
    observed_dim = 6  # number of observed variables
    # Construct M as a product of two matrices to force low rank.
    A = np.random.randn(observed_dim, 10)      # (6 x 2)
    B = np.random.randn(10, X_latent.shape[1])   # (2 x 9)
    M = A @ B  # Resulting mixing matrix of shape (6, 9) with rank at most 2.
    
    # Mix the latent signals to generate the observed data.
    X_mixed = X_latent @ M.T  # shape (steps, observed_dim)
    
    # Initialization parameters for FlowDecomposition.
    init_params = {
        "input_dim": observed_dim,  # now 6 instead of 4
        "proj_dim": 1,
        "n_components": 3,          # expecting two independent latent dynamics
        "num_delays": 3,
        "delay_step": 5,
        "subtract_corr": False,
        "device": "cpu",
        "optimizer": "Adagrad",
        "learning_rate": 0.01,
        "random_state": None
    }
    
    # Fit parameters.
    fit_params = {
        "sample_len": 150,
        "library_len": 700,
        "exclusion_rad": 10,
        "theta": 5,
        "tp": 20,
        "epochs": 100,
        "num_batches": 10,
        "beta": 0,
        "tp_policy": "range",
        "loss_mask_size": None
    }
    
    fd = FlowDecomposition(**init_params)
    fd.fit(X_mixed, **fit_params)

if __name__ == "__main__":
    test_flow_decomposition()


In [13]:

# Create dummy data: a time series with 100 points and 1 feature.
X = np.linspace(0, 99, 4000).reshape(1000,4).T.reshape(1000,4)
print(X)

# Initialization parameters dictionary.
init_params = {
    "input_dim": 4,
    "proj_dim": 2,
    "n_components": 1,
    "num_delays": None,
    "delay_step": None,
    "subtract_corr": False,
    "device": "cpu",
    "optimizer": "Adam",
    "learning_rate": 0.01,
    "random_state": None
}

# Fit parameters dictionary.
fit_params = {
    "sample_len": 50,
    "library_len": 700,
    "exclusion_rad": 1,
    "theta": 3,
    "tp": 20,
    "epochs": 100,
    "num_batches": 20,
    "beta": 1,
    "tp_policy": "range",
    "loss_mask_size": None
}

fd = FlowDecomposition(**init_params)
fd.fit(X, **fit_params)


[[ 0.          0.09902476  0.19804951  0.29707427]
 [ 0.39609902  0.49512378  0.59414854  0.69317329]
 [ 0.79219805  0.89122281  0.99024756  1.08927232]
 ...
 [97.91072768 98.00975244 98.10877719 98.20780195]
 [98.30682671 98.40585146 98.50487622 98.60390098]
 [98.70292573 98.80195049 98.90097524 99.        ]]
Epoch 1/100, Loss: 1.7310, ccm_loss: 0.9630, h_norm_loss: 0.7680
Epoch 2/100, Loss: 1.7386, ccm_loss: 0.9749, h_norm_loss: 0.7637
Epoch 3/100, Loss: 1.7245, ccm_loss: 0.9641, h_norm_loss: 0.7604
Epoch 4/100, Loss: 1.7135, ccm_loss: 0.9564, h_norm_loss: 0.7571
Epoch 5/100, Loss: 1.7222, ccm_loss: 0.9683, h_norm_loss: 0.7539
Epoch 6/100, Loss: 1.7088, ccm_loss: 0.9590, h_norm_loss: 0.7498
Epoch 7/100, Loss: 1.7037, ccm_loss: 0.9579, h_norm_loss: 0.7458
Epoch 8/100, Loss: 1.7050, ccm_loss: 0.9630, h_norm_loss: 0.7420
Epoch 9/100, Loss: 1.6840, ccm_loss: 0.9463, h_norm_loss: 0.7377
Epoch 10/100, Loss: 1.6878, ccm_loss: 0.9528, h_norm_loss: 0.7350
Epoch 11/100, Loss: 1.6824, ccm_loss: